In [3]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/Test

/content/drive/MyDrive/Test


Import Libraries

In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

Load ascii text and convert to lowercase

In [6]:
filename="data.txt"
raw_text= open(filename, 'r', encoding= 'utf-8').read()
raw_text= raw_text.lower()

In [7]:
raw_text[0:100]

"project gutenberg's alice's adventures in wonderland, by lewis carroll\n\nthis ebook is for the use of"

In [7]:
#To Do
#clean data set to remove unwanted characters

In [8]:
#create mapping of unique chars to integers\n
chars= sorted(list(set(raw_text)))
print(chars)
char_to_int= dict((c,i) for i,c in enumerate(chars))

['\n', ' ', '!', '"', '#', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
print(char_to_int)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, "'": 7, '(': 8, ')': 9, '*': 10, ',': 11, '-': 12, '.': 13, '/': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, ';': 26, '?': 27, '@': 28, '[': 29, ']': 30, '_': 31, 'a': 32, 'b': 33, 'c': 34, 'd': 35, 'e': 36, 'f': 37, 'g': 38, 'h': 39, 'i': 40, 'j': 41, 'k': 42, 'l': 43, 'm': 44, 'n': 45, 'o': 46, 'p': 47, 'q': 48, 'r': 49, 's': 50, 't': 51, 'u': 52, 'v': 53, 'w': 54, 'x': 55, 'y': 56, 'z': 57}


Getting the details of the dataset

In [10]:
n_chars= len(raw_text)
n_vocab= len(chars)
print("Total Characters : ", n_chars)
print("Total Vocab(unique characters): ", n_vocab)

Total Characters :  163780
Total Vocab(unique characters):  58


In [12]:
#To Do
#change the value of seq_length and chechk the training data set
#can also add text data set

In [11]:
raw_text[0:5]

'proje'

In [12]:
#prepare the dataset of input to output pairs encoded as integer
seq_length= 100 #can be changed
dataX= []
dataY= []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out= raw_text[i + seq_length]
  dataX.append([char_to_int[char]for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns= len(dataY)
print("Total patterns: ", n_patterns)


Total patterns:  163680


In [13]:
#checikng DataX and DataY
print(dataX[1])
print(dataY[1])

[49, 46, 41, 36, 34, 51, 1, 38, 52, 51, 36, 45, 33, 36, 49, 38, 7, 50, 1, 32, 43, 40, 34, 36, 7, 50, 1, 32, 35, 53, 36, 45, 51, 52, 49, 36, 50, 1, 40, 45, 1, 54, 46, 45, 35, 36, 49, 43, 32, 45, 35, 11, 1, 33, 56, 1, 43, 36, 54, 40, 50, 1, 34, 32, 49, 49, 46, 43, 43, 0, 0, 51, 39, 40, 50, 1, 36, 33, 46, 46, 42, 1, 40, 50, 1, 37, 46, 49, 1, 51, 39, 36, 1, 52, 50, 36, 1, 46, 37, 1]
32


Transform the list of input sequences into the form [samples, time steps, features] that is expected by an
LSTM network and rescale the integers to the range [0,1] to make the patterns easier to learn by the LSTM
network that uses the sigmoid activation function by default.


In [14]:
#reshape X to be[samples, time steps, fetures]
X= numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize - rescaling the integer values
X= X/ float(n_vocab)
print(X)

[[[0.81034483]
  [0.84482759]
  [0.79310345]
  ...
  [0.01724138]
  [0.79310345]
  [0.63793103]]

 [[0.84482759]
  [0.79310345]
  [0.70689655]
  ...
  [0.79310345]
  [0.63793103]
  [0.01724138]]

 [[0.79310345]
  [0.70689655]
  [0.62068966]
  ...
  [0.63793103]
  [0.01724138]
  [0.55172414]]

 ...

 [[0.87931034]
  [0.79310345]
  [0.01724138]
  ...
  [0.79310345]
  [0.79310345]
  [0.72413793]]

 [[0.79310345]
  [0.01724138]
  [0.67241379]
  ...
  [0.79310345]
  [0.72413793]
  [0.86206897]]

 [[0.01724138]
  [0.67241379]
  [0.62068966]
  ...
  [0.72413793]
  [0.86206897]
  [0.22413793]]]


 Convert the output values (single characters converted to integers) into a one hot encoding

In [15]:
from keras.src.utils import np_utils
#one hot encode the output variable
y= np_utils.to_categorical(dataY)
print(y)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [24]:
#To Do
#change the configuartion of the model(adding more layers, change LSTM units, change dropout values, optimizer function)

 Define the LSTM model

In [16]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]))) #it can have 1 or more training samples
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#define the check point
filepath= "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint= ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list= [checkpoint]

 Fitting the model to data

In [27]:
#change the hyperparameter values and train model
epochs= 2
batch_size= 128

In [28]:
model.fit(X,y, epochs= epochs, batch_size=batch_size, callbacks= callbacks_list)

Epoch 1/2
1279/1279 [==============================] - ETA: 0s - loss: 2.9752
Epoch 1: loss improved from inf to 2.97520, saving model to weights-improvement-01-2.9752.hdf5
1279/1279 [==============================] - 1136s 886ms/step - loss: 2.9752
Epoch 2/2


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1279/1279 [==============================] - ETA: 0s - loss: 2.7998
Epoch 2: loss improved from 2.97520 to 2.79976, saving model to weights-improvement-02-2.7998.hdf5
1279/1279 [==============================] - 946s 739ms/step - loss: 2.7998


 Generate Text with the trained LSTM model

In [17]:
#load the networks weights
filename= "weights-improvement-02-2.7998.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
int_to_char= dict((i,c)for i,c in enumerate(chars))

In [19]:
#generate a random seed
print(len(dataX))
start= numpy.random.randint(0, len(dataX)-1)
print(start)
pattern= dataX[start]#dataX contains list of patterns
print("seed:")
print("\"",''.join([int_to_char[value]for value in pattern]),"\"")

163680
157484
seed:
"  the problem.

1.f.4.  except for the limited right of replacement or refund set forth
in paragraph  "


In [20]:
#generate Characters
length= 10
final=[]
for i in range(length):
  #reshaping the seed sequence before passing it into the LSTM model
  x= numpy.reshape(pattern, (1,len(pattern),1))
  #print(x)
  # normlaizing the integer values
  x= x/ float(n_vocab)
  #print (x)
  #making prediction
  prediction= model.predict(x, verbose= 0)
  #get the predicted value with maximum probability
  index= numpy.argmax(prediction)

  #convert the predicted integer to char
  result= int_to_char[index]
  #print(reuslt)
  final.append(result)
  #Adding the predicted character to the sequence sequence
  pattern.append(index)
  #removing the first character from the seed sequence
  pattern = pattern[1:len(pattern)]

print(final)

['t', 'h', ' ', 't', 'h', 'e', ' ', ' ', 'a', 'n']
